In [ ]:
import numpy as np
import scipy.stats 
import matplotlib.pyplot as plt
#%matplotlib notebook
plt.rcParams['figure.dpi'] = 100

In [ ]:
N = 20

M = np.random.rand(N,2*N)
n_col = M.shape[1]
n_row = M.shape[0]

x_col = np.ones(n_col)
x_row = np.ones(n_row)

x_col = np.random.randint(2, size=n_col)
x_row = np.random.randint(2, size=n_row)

print(x_col)
print(x_row)

In [ ]:
def F(x_col,x_row,M):
    M_a = x_row.reshape(-1,1)*M*x_col.reshape(1,-1) 
    M_b = (1-x_row).reshape(-1,1)*M*(1-x_col).reshape(1,-1)
    return np.linalg.norm(M_a,'nuc')+np.linalg.norm(M_b,'nuc')

In [ ]:
def subdifferential(x_col,x_row,M):
    M_a = x_row.reshape(-1,1)*M*x_col.reshape(1,-1) 
    M_b = (1-x_row).reshape(-1,1)*M*(1-x_col).reshape(1,-1)
    #print(M_a-M_b)
    #print(M_a);print(M_b)
    U_a, s, Vh_a = np.linalg.svd(M_a, full_matrices=False)
    U_b, s, Vh_b = np.linalg.svd(M_b, full_matrices=False)
    
    G = (Vh_a.T@U_a.T-Vh_b.T@U_b.T)
    #print(G)

    d_row = np.diag((M*x_col.reshape(1,-1))@G)
    d_col = np.diag(G@(x_row.reshape(-1,1)*M))
    
    return (d_col,d_row)

Test derivative

In [ ]:
d_col,d_row = subdifferential(x_col,x_row,M)
print(d_col)
print(d_row)

In [ ]:
eps = 5e-4
xp_row=np.ones_like(x_row)*eps
xp_col=np.ones_like(x_col)*eps

In [ ]:
F_prime = F(x_col+xp_col,x_row+xp_row,M)
F_est = F(x_col,x_row,M)+d_col@xp_col+d_row@xp_row

In [ ]:
F_est

In [ ]:
F_prime

In [ ]:
F(x_col,x_row,M)

In [ ]:
d_col

In [ ]:
d_row

In [ ]:
L = 20

gamma = 0.1

Fs = np.zeros(L)
x_cols = np.zeros([L,n_col])
x_rows = np.zeros([L,n_row])

In [ ]:
Fs[0]=F(x_col,x_row,M)
x_cols[0]=x_col
x_rows[0]=x_row
for l in range(1,L):
    d_col,d_row = subdifferential(x_col,x_row,M)
    x_col = x_col-gamma*d_col
    x_row = x_row-gamma*d_row
    np.clip(x_col,0,1,out=x_col)
    np.clip(x_row,0,1,out=x_row)
    Fs[l]=F(x_col,x_row,M)
    x_cols[l]=x_col
    x_rows[l]=x_row

In [ ]:
plt.plot(Fs);

In [ ]:
plt.plot(x_cols);

To aviod that all are accumulated in one part we add an regularization term

We substract a vaule if they are groupd in two parts. If all are in one part, then there the regularization term is zero

$$
g(x) = - \sum x_i \sum 1-x_i  
$$

for both the rows and the columns

Spoiler: This is actually not a very good regularization unless I did some error when computing the derivative

In [ ]:
def F_reg(x_col,x_row):
    return -np.sum(x_col)*(np.sum(1-x_col))-np.sum(x_row)*(np.sum(1-x_row))

def F_deriva(x_col,x_row):
    d_row = (2*np.sum(x_row)-np.size(x_row))*np.ones_like(x_row)
    d_col = (2*np.sum(x_col)-np.size(x_col))*np.ones_like(x_col)
    return (d_col,d_row)

In [ ]:
eps = 5e-4
xp_row=np.ones_like(x_row)*eps
xp_col=np.ones_like(x_col)*eps
d_col,d_row = F_deriva(x_col,x_row)
F_prime = F_reg(x_col+xp_col,x_row+xp_row)
F_est = F_reg(x_col,x_row)+d_col@xp_col+d_row@xp_row
print(F_prime)
print(F_est)
print(F_reg(x_col,x_row))

In [ ]:
L = 20

gamma = 0.2

nu = 1e-2 # for regularization

Fs = np.zeros(L)
x_cols = np.zeros([L,n_col])
x_rows = np.zeros([L,n_row])

In [ ]:
Fs[0]=F(x_col,x_row,M)+gamma*F_reg(x_col,x_row)
x_cols[0]=x_col
x_rows[0]=x_row
for l in range(1,L):
    d_col,d_row = subdifferential(x_col,x_row,M)
    d_reg_col,d_reg_row = F_deriva(x_col,x_row)
    x_col = x_col-gamma*(d_col+nu*d_reg_col)
    x_row = x_row-gamma*(d_row+nu*d_reg_row)
    np.clip(x_col,0,1,out=x_col)
    np.clip(x_row,0,1,out=x_row)
    Fs[l]=F(x_col,x_row,M)+gamma*F_reg(x_col,x_row)
    x_cols[l]=x_col
    x_rows[l]=x_row

In [ ]:
plt.plot(Fs);

In [ ]:
plt.plot(x_cols);

In [ ]:
x_row